# Expérience avec `pyrealb`

_[This notebook in English](https://mybinder.org/v2/gh/lapalme/pyrealb-jupyter/HEAD?labpath=pyrealb-en.ipynb)_

Ce _notebook Jupyter_ notebook montre quelques exemples de réalisation de texte avec **pyrealb**.  

**À l'ouverture, choisissez l'item de menu `Run/Run all cells`**

Vous pouvez aussi l'utiliser dans votre browser (en passant par Binder) avec [ce lien](https://mybinder.org/v2/gh/lapalme/pyrealb-jupyter/HEAD?labpath=pyrealb-en.ipynb). 

**pyrealb** est un système qui permet de réaliser des phrases en anglais ou en français à partir d'une structure JavaScript créée par des appels à des fonctions. **pyrealb** gère automatiquement la conjugaison ou déclinaison ainsi que la majorité des accords entre les constituants. 

La syntaxe des fonctions a été conçue pour ressembler à celle qui est utilisée dans les grammaires en constituants: des terminaux sont intégrés à des syntagmes, chacun d'eux pouvant ensuite être modifié par des options qui sont spécifiées par des appels de *méthodes* ou fonctions internes aux structures ainsi créées. Une notation sous forme de dépendances syntaxiques est aussi disponible.

Une des flexibilités intéressantes est le fait qu'on peut obtenir des variantes (p.ex. négatives ou passives) d'une même phrase en spécifiant des options. Nous n'expliquons pas ici toutes les options ni toutes les valeurs possibles de leurs paramètres. Pour ceci, il faut consulter la  [documentation **pyrealb** ](http://www.iro.umontreal.ca/~lapalme/pyrealb/documentation.html).

Ce notebook *Jupyter* présente brièvement la syntaxe utilisée par **pyrealb** à l'aide d'exemples simples mais aussi avec quelques exemples plus complexes.  Vous pouvez modifier les exemples and voir leur nouvelle réalisation. Lorsqu'une expression est changée, on peut lancer son évaluation avec (shift-return) ou en cliquant sur un triangle orienté vers la droite à gauche de la cellule. Il faut se rappeler que, contraitement à un tableur, le changement d'une expression **n'a pas** un effet immédiat sur les expressions qui en dépendent. On peut forcer leur réévaluation en cliquand sur un triangle orienté vers la droite avec une flèche vers le bas (_Execute cell and below_). 

*Nota bene*: losrque **pyrealb** détecte une erreur dans une spécification la réalisation apparaît avec un mot entre double crochets `[[...]]`, un message est aussi affiché avant la réalisation.

## Chargement de `pyrealb`
Il faut tout d'abord importer les fonctions du package  `pyrealb` qui aura déjà dû être installé en utilisant `pip`. On indique ensuite que nous allons réaliser du texte en français.

In [1]:
from pyrealb import *
loadFr()        # indiquer que la réalisation se fera en français
pyrealb_version # vérifier la version de pyrealb qui devrait être '3.0.0'

'3.1'

## Création et réalisation d'un premier mot

On appelle un constructeur pour créer une structure Python, par exemple pour créer un nom

In [2]:
N("chat")

Cet appel a bien construit une structure Python intéressante, mais pour en obtenir la réalisation sous forme de chaîne de caractères, il faut appeler `str(..)` (`print(..)` le fait implicitement) ou `.realize()` sur l'objet. 

Pour simplifier la notation dans les prochains exemples, nous définissons la fonction `realiser()` qui accepte un nombre quelconque de paramètres, qui réalise chacun d'un et qui combine leurs réalisations en une chaine avec un séparateur (une virgule, par défaut).

In [3]:
def realiser(*exps,sep=', '):
    return sep.join(exp.realize() for exp in exps)

qui peut être appelé comme suit:

In [4]:
realiser(N('chat'),N("chien"),N("papillon"))

'chat, chien, papillon'

## Création de terminaux
On appelle le constructeur en lui donnant en paramètre la forme de base du mot:
* singulier pour un article, un nom ou un adjectif

In [5]:
realiser(D("un"),
         N("chat"),
         A("gris"))

'un, chat, gris'

* infinitif pour un verbe qui est par défaut conjugué au présent, 3e personne du singulier

In [6]:
realiser(V("aimer"))

'aime'

* première personne du singulier pour un pronom, qui est décliné par défaut à la 3e personne du singulier

In [7]:
realiser(Pro("je"))

'il'

* adverbe, préposition, conjonction et *canned (quoted) text*, on spécifie la forme qui ne sera pas fléchié à la réalisation

In [8]:
realiser(Adv("tellement"),
         P("de"),
         C("ou"),
         Q("Wow!!"))

'tellement, de, ou, Wow!!'

* date: actuelle si aucun paramètre, sinon une date en format ISO. On verra plus tard comment chosir les champs à réaliser 

In [9]:
realiser(DT("2020-12-25"),
         DT(),
         sep="; ")

'le vendredi 25 décembre 2020 à minuit; le dimanche 27 octobre 2024 à 14 h 40 min 14 s'

* nombre correspondant à la valeur numérique donnée en paramètre

In [10]:
realiser(NO(123),
         NO(45678))

'123, 45\xa0678'

## Modifications à l'aide d'options: _[fonctions d'un terminal](http://www.iro.umontreal.ca/~lapalme/pyrealb/documentation.html#optionsFr)_:
* nombre: singulier `.n("s")`, pluriel `.n("p")`
* genre: masculin `.g("m")`, féminin `.g("f")`

In [11]:
realiser(D("un").g("f"), 
         N("lapin").n("p"), 
         A("gris").g("f").n("p"))

'une, lapins, grises'

* personne : première `.pe(1)`, deuxième `.pe(2)` ou troisième `.pe(3)`, à combiner avec le nombre
* temps: passé simple `.t("ps")`, passé composé `.t("pc")`, futur `.t("f")`, ... 

In [12]:
realiser(Pro("je").pe(2).n("p"), 
         V("aimer").t("ps").pe(2).n("p"),
         Pro("moi").pe(3), 
         V("finir").t("f").pe(1).n("p"))

'vous, aimâtes, lui, finirons'

* formatage de dates

In [13]:
realiser(DT().dOpt({"hour":False,"minute":False,"second":False,"nat":True}),
         DT("2020-12-25").dOpt({"hour":False,"minute":False,"second":False,"nat":False}),
         sep="; ")

'le dimanche 27 octobre 2024; vendredi 25/12/2020'

* formatage de nombres

In [14]:
realiser(NO(123).dOpt({"nat":True}),
         NO(15).dOpt({"ord":True}),
         NO(3.141592).dOpt({"mprecision":4}))

'cent vingt-trois, quinzième, 3,1416'

* sortie HTML

In [15]:
realiser(D('un'),
         N("chat").tag("a",
                       {"href":"https://fr.wikipedia.org/wiki/Chat",
                        "target":"_blank"}),
         A("gris").tag("i"))

'un, <a href="https://fr.wikipedia.org/wiki/Chat" target="_blank">chat</a>, <i>gris</i>'

## Création de syntagmes
Les syntagmes sont obtenus par composition d'appels à des terminaux ou à d'autres syntagmes.
* *syntagme nominal* (`NP`) à l'intérieur duquel le genre et le nombre du premier nom sont propagés aux autres composants, la position des adjectifs par rapport au nom est donnée par le lexique
* *syntagme verbal* (`VP`) contenant souvent un syntagme nominal comme complément direct
* *phrase* (`S`) qui combine typiquement (mais pas seulement) un syntagme nominal et un syntagme verbal. Le premier syntagme nominal est considéré comme le sujet de la phrase avec lequel le verbe du premier syntagme verbal sera accordé. Par défaut, `S` à la racine met la première lettre en majuscule et ajoute un point à la fin de la phrase.

Nous créons deux syntagmes nominaux, un syntagme verbal et une phrase qui seront utilisés dans des expressions ultérieures. 

In [42]:
np1 = NP(D("le"),
         N("souris").n("p"),
         A("petit"),
         A("gris"))
np2 = NP(D("le"), 
         N("orange"))
s = S(np1,
      VP(V("manger").t("f"), 
          np2))

Maintenant affichons leur réalisations

In [43]:
realiser(np1,np2,s)

"les petites souris grises, l'orange, Les petites souris grises mangeront l'orange. "

### Modifications de syntagme avec des options, i.e. [fonctions qui sont des propriétés des objets Phrase](http://rali.iro.umontreal.ca/JSrealB/current/documentation/user.html?lang=fr#npMod)

Une caractéristique intéressante d'une phrase est le fait qu'elle peut être modifiée. Par exemple, une structure de phrase affirmative peut être réalisée comme une négative, une passive ou une question.  Plus d'un millier de structures différentes peuvent être obtenues à partir d'une seule structure affirmative.

Par exemple, pour réaliser une version négative et passive de la phrase précédente:

In [18]:
realiser(S(np1,
           VP(V("manger").t("f"), 
              np2)).typ({"neg":True,"pas":True}))

"L'orange ne sera pas mangée par les petites souris grises. "

Comme une option peut modifier la structure interne d'un syntagme lors de sa réalisation, c'est aussi vrai pour un terminal, il est préférable de créer une copie de l'objet avant de le réaliser, lorsqu'on prévoit le réutiliser.

**Ce _problème_ est exacerbé dans ce  _notebook Jupyter_ car une cellule peut être évaluée, donc réalisée, plusieurs fois modifiant ainsi la structure à chaque fois.**

Une façon simple de créer une copie d'une expression est de définir une fonction qui retournera une nouvelle expression à chaque appel. Cette façon de procéder a l'avantage de permettre de paramétrer l'expression. Pour notre variable `np1`, elle sera paramétrée par le nombre, singulier par défaut. Par convention, nous suffixons le nom par `_f` pour nous rappeler que c'est une fonction.

In [19]:
def np1_f(n="s"):
  return NP(D("le"),
            N("chat").n(n),
            A("petit"),
            A("noir"))

Cette fonction peut être appelée plusieurs fois, créant une nouvelle expression à chaque appel

In [20]:
realiser(np1_f("p"),np1_f())

'les petits chats noirs, le petit chat noir'

Comme ce type de fonction se limite souvent à retourner une valeur, on peut utiliser des _lambda_. On peut définir deux fonctions pour les syntagmes précédents et réaliser deux nouvelles phrases.

In [21]:
np2_f = lambda n = "s": NP(D("le"),N("orange").n(n))
s_f = lambda n = "s" : S(np1_f(n),VP(V("manger"),np2_f(n)))
realiser(s_f("p"),s_f().typ({"neg":True,"pas":True}),sep="|| ")

"Les petits chats noirs mangent les oranges. || L'orange n'est pas mangée par le petit chat noir. "

Pour simplifier la notation pour les prochains exemples de transformations, nous définissons la fonction suivante qui évalue la fonction pour créer l'expression sur laquelle est appliquée la transformation.

In [22]:
def afficher(ph_f,mod_typ={}):
    return ph_f().typ(mod_typ).realize()

### Phrase originale:

* sa réalisation

In [23]:
afficher(s_f)

"Le petit chat noir mange l'orange. "

* sa négation

In [24]:
afficher(s_f,{"neg":True})

"Le petit chat noir ne mange pas l'orange. "

* sa négation au passif introduisant l'auxiliaire être dont l'attribut doit s'accorder avec le sujet 

In [25]:
afficher(s_f,{"neg":True,"pas":True})

"L'orange n'est pas mangée par le petit chat noir. "

* question sur le sujet du verbe

In [26]:
afficher(s_f,{"int":"wos"})

"Qui mange l'orange? "

* question sur l'objet du verbe

In [27]:
afficher(s_f,{"int":"wad"})

"Qu'est-ce que le petit chat noir mange? "

### Variante avec une subordonnée

La subordonnée est créée en ajoutant un `SP` comme dernier constituant de `np2_f`. Ceci a permis d'imbriquer la subordonnée à l'intérieur du `NP`. 

In [28]:
sp_f = lambda : S(np2_f().add(SP(Pro("que"),
                                 np1_f(),
                                 VP(V("manger").t("pc")))),
                 VP(V("être"),
                    A("mûr")).t("ps"))

* la structure résultante montrée en tant que code source
``` python
S(NP(D('le'),
     N('orange').n("s"),
     SP(Pro('que'),
        NP(D('le'),
           N('chat').n("s"),
           A('petit'),
           A('noir')),
        VP(V('manger').t("pc")))),
  VP(V('être'),
     A('mûr')).t("ps"))
```

* sa réalisation

In [29]:
afficher(sp_f)

"L'orange que le petit chat noir a mangée fut mûre. "

* sa négation

In [30]:
afficher(sp_f,{"neg":True})

"L'orange que le petit chat noir a mangée ne fut pas mûre. "

### Syntagme coordonné

`CP` permet de réaliser une liste de syntagmes où tous les éléments sont séparés par une virgule, sauf les deux derniers qui sont séparés par une conjonction qui est spécifiée à l'intérieur (souvent comme premier paramètre) du constructeur:

In [31]:
cp_f = lambda: CP(C("et"),
                  NP(D("le"),N("éléphant")),
                  NP(D("le"),N("souris")),
                  NP(D("un"),N("loup"))) 

* une utilisation dans une phrase

In [32]:
afficher(lambda: S(cp_f(), 
                   VP(V("arriver").t("pc"))))

"L'éléphant, la souris et un loup sont arrivés. "

* on peut y ajouter des éléments, ici un nouveau `NP` contenant un nombre en toutes lettres

In [33]:
afficher(lambda: S(cp_f().add(NP(NO(25).nat(True),
                            N("chien"))),
                   VP(V("arriver").t("pc"))))

"L'éléphant, la souris, un loup et vingt-cinq chiens sont arrivés. "

### Pronominalisation

Afin d'alléger le texte, il est souvent intéressant de remplacer un syntagme nominal par un pronom.
Soient les deux constituants suivants:

In [34]:
garcon_f = lambda: NP(D("le"),N("garçon"),A("beau"))
femme_f = lambda: NP(D("un"),N("femme"),A("intelligent"))

Construisons une phrase initiale avec un verbe au passé composé

In [35]:
afficher(lambda: S(garcon_f(),
                   VP(V("aimer").t("pc"),
                      femme_f())))

'Le beau garçon a aimé une femme intelligente. '

Pronominalisons le sujet

In [36]:
afficher(lambda: S(garcon_f().pro(),
                   VP(V("aimer").t("pc"),
                      femme_f())))

'Il a aimé une femme intelligente. '

Pronominalisons aussi l'objet. On remarque que le pronom a été placé avant le verbe et que le participe passé avec *avoir* a été accordé correctement avec son complément d'objet direct qui maintenant le précède.

In [37]:
afficher(lambda: S(garcon_f().pro(),
                   VP(V("aimer").t("pc"),
                      femme_f().pro())))

"Il l'a aimée. "

## Notation en dépendances

Pour réaliser des phrases, il est aussi possible d'utiliser une notation en [syntaxe de dépendances](https://en.wikipedia.org/wiki/Dependency_grammar). 

Une dépendance est créée avec une fonction qui indique le nom de la relation: `root`, `subj` (sujet) , `det` (determiner) `comp` (complément), `mod` (modificateur). Leur premier paramètre est un `Terminal` qui constitue la tête de cette dépendance. Les autres paramètres, s'il y en a, sont les dépendances associées à la tête. 

Plutôt que de combiner des syntagmes pour créer une phrase comme on l'a montré précédemment, on construit une structure par des appels correspondant des noms de relation indiquant le rôle de cette dépendance dans la phrase. Le premier paramètre étant la tête de la dépendance. **pyrealb** utilise cette information sur les rôles pour effectuer les accords. 

Dans l'exemple suivant, le sujet au pluriel forcera le pluriel du déterminant et du verbe. Dans le complément, le genre féminin de souris est propagé au modificateur, l'adjectif `blanc`.

In [38]:
afficher(lambda:root(V("manger").t("i"),
                     subj(N("chat").n("p"),
                          det(D("le"))),
                     comp(N("souris"),
                          det(D("un")),
                          mod(A("blanc")))))

'Les chats mangeaient une souris blanche. '

Les phrases coordonnées sont construites à l'aide de la fonction `coord` qui a comme tête la conjonction de coordination et, comme dépendants, des relations qui doivent toutes être du même type. Par exemple;

In [39]:
afficher(lambda:root(V("manger"),
                     coord(C("et"),
                          subj(N("garçon"),det(D("le"))),
                          subj(N("fille"),det(D("le")))),
                     coord(C("ou"),
                          comp(N("soupe"),det(D("le"))),                  
                          comp(N("légume").n("p"),det(D("un"))),
                          comp(N("poulet"),det(D("le"),
                                               mod(P("de")).pos("pre"))))),{"pas":True,"neg":True})

'La soupe, des légumes ou du poulet ne sont pas mangés par le garçon et la fille. '

**pyrealb** détermine l'ordre des mots dans une phrase: `det` et `subj` apparaissent avant la tête, alors que `comp` et `mod` viennent après. Cet ordre par défaut peut être changé en spécifiant `.pos()` avec paramètre `pre` ou `post` comme dans l'exemple précédent où on voulait que `de` apparaisse avant `le`. En cas d'égalité de position, on utilise la position relative dans l'expression.

## Conclusion

Ces quelques expressions ont illustré certaines possibilités de **pyrealb** pour la réalisation de phrases en français. Plusieurs accords sont effectués automatiquement ainsi que l'élision. Une fois établie la structure de base d'une phrase, il est possible d'en obtenir de multiples variantes (négation, passivation ou interrogation) en ne spécifiant que des options au niveau de la phrase. 

Vous pouvez voir d'autres [démonstrations](https://github.com/lapalme/pyrealb/tree/main/demos).

[Guy Lapalme](mailto:lapalme@iro.umontreal.ca)